In [ ]:
%run stdPackages.ipynb # this imports a lot of useful packages 
import base

# The ```lpBlock``` class - Part I: Introduction and model structure

The ```lpBlock``` class is used to define linear programming (LP) models that can be passed to the solver from ```scipy.optimize.linprog```. The essential functionality, compared to interacting directly with ```linprog``` using stacked arrays is that we formulate the model using variable names and indices. This notebook takes you through the main steps of the ```lpBlock``` method. We draw on a relatively simple model to illustrate the syntax along the way.

The note proceeds as follows:
>[1. Augmented form LP](#AugLP): Defines mathematical format used in ```scipy.optimize.linprog```.  
>[2. Model example](#ModelExample): Defines an example to draw on throughout the rest of the notebook; provides a "manual" approach to setting up the augmented-form LP for this model.  
>[3. Model specification with lpBlock](#ModelLP): Shows how to use the class ```lpBlock``` to create the model structure from section 2.  
>> [3.1. Fundamentals](#ModelLP_Fundamentals): Provides an overview of the fundamental building blocks that we use to create the model based on pandas indices and variable/constraint names.  
>> [3.2-3.5. Model components](#ModelLP_componentsStart): Show how to create model components (```c, l, u, b_eq, b_ub, A_eq, A_ub```).  
>> [3.6. Code summary](#ModelLP_summary): Shows an overview of the code required to specify the model using ```lpBlock```.

[**Part II**](./lpCompiler_PartII.ipynb) shows how the ```lpBlock``` class compiles the arguments and passes it to the solver.

## 1. Augmented form linear programming <a id='AugLP'></a>

The algorithm in ```scipy.optimize.linprog``` solves what we will refer to as an "augmented form" of the linear program (as opposed to the "standard form" that does not feature explicit equality constraints):

$$\begin{align} 
    &\min_{x} \ c^T\cdot x \tag{1A}\\ 
    &A_{ub}\times x \leq b_{ub} \tag{1B}\\ 
    &A_{eq}\times x  = b_{eq} \tag{1C}\\ 
    &l\leq x\leq u, \tag{1D}
\end{align}$$
where: 
* $x$ is the vector of choice variables of length ($N$).
* $c, l, u$ are coefficient vectors of the same length ($N$).
* $b_{eq}, b_{ub}$ are coefficient vectors of lengths $N_{eq}, N_{ub}$, 
* and $A_{eq}, A_{ub}$ are coefficient matrices of sizes $(N_{eq}\times N)$ and $(N_{ub} \times N)$ respectively.

We will generally refer to the inequality constraints $l\leq x \leq u$ as *domain constraints*, whereas the constraints that combine multiple variables as *variational constraints* (1B-1C). 

At its core, the ```scipy.optimize.linprog``` takes inputs in the form of arrays (e.g. ```np.arrays```), which means that we have to be careful with the ordering of variables and constraints: The $n$'th element in $c$ represents the same element as the $n$'th elements in $l,u$ and the $n$'th column vectors in $A_{ub}, A_{eq}$. Thus, when solving a model with many different types of constraints and variables, the task of constructing suitable vectors / matrices becomes quite cumbersome. The main objective of the ```lpBlock``` compiler is to make it a bit easier to formulate this type of LP models. 

## 2. Model example <a id='ModelExample'></a>

In the following, we will use the model from ```mBasicInt``` to illustrate the way the model works. This model is represented by the optimization problem

$$\begin{align}
\min_{ E_{id,h}, D_{c,h}} -W &= -\sum_{h} \left[\sum_c\left(\text{MWP}_c\cdot D_{c,h}\right) - \sum_{id}\left(\text{mc}_{id}\cdot E_{id,h}\right) \right] \tag{2A}\\ 
\text{s.t. }\sum_c D_{c,h} &= \sum_{id} E_{id,h}, \qquad \forall h \tag{2B}\\
D_{c,h} &\in[0, \overline{D}_{c}], \qquad \forall c,h \tag{2C}\\ 
E_{id,h}&\in[0, q_{id,h}], \qquad q_{id,h} = q_{id} \cdot \gamma_{id,h}, \qquad \forall id, h \tag{2D}
\end{align}$$

This system of equations represents maximizing the economic surplus from the electricity market where (*the problem is defined as a minimization problem to make it easier to compare to the augmented-form*):
* Consumers (indexed $c$) have a marginal willingness to pay of $\text{MWP}_c$ per GJ of electricity. They consume $D_{c,h}$ in hour $h$, which is non-negative and below a maximum load $\overline{D}_c$. 
* The demand is met by a set of individual generators (indexed $id$) that generates electricity at a marginal cost of $\text{mc}_{id}$. 
* Each plant produces $E_{id,h}$ GJ of electricity in hour $h$, which has to be non-negative and cannot exceed their installed capacity $q_{id,h}$. Some plants (like wind and PV) have capacity limits that vary hourly according to some exogenous process ($\gamma_{id,h}\in[0,1]$ captures this.)

#### A manual approach to set up the augmented-form LP 

Let us, for simplicity, assume that there are two elements in all of the model sets. For parameter values, assume:

In [ ]:
MWP  = pd.Series([10, 20], index = pd.Index(['Consumer 1','Consumer 2'], name = 'c'))
D̄    = pd.Series([0.5, 0.5], index = pd.Index(['Consumer 1','Consumer 2'], name = 'c'))
mc   = pd.Series([15, 5], index = pd.Index(['Conv. plant','Wind turbine'], name = 'id'))
q_id = pd.Series([0.5, 0.5], index = pd.Index(['Conv. plant', 'Wind turbine'], name = 'id'))
γ    = pd.Series([1, 1, 1, 0.5], index = pd.MultiIndex.from_tuples([('Conv. plant',1), ('Wind turbine',1),
                                                                    ('Conv. plant',2), ('Wind turbine', 2)],
                                                                   names = ['id','h']))
q    = q_id * γ

In this model, we optimize by solving for generation variable ($E_{id,h}$) and demand variables ($D_{c,h}$). In the augmented-form LP notation, this means that we can write the **x** vector by stacking the two:
$$\begin{align}
    \mathbf{x} = \begin{pmatrix} \mathbf{E} \\ \mathbf{D} \end{pmatrix} = \begin{pmatrix} E_{Conv, 1} \\ E_{Wind, 1} \\ E_{Conv, 2} \\ E_{Wind, 2} \\ D_{C1, 1} \\ D_{C2, 1} \\ D_{C1, 2} \\ D_{C2,2} \end{pmatrix}, \tag{3A}
\end{align}$$
where we have used some notational abbreviations for convenience.

The coefficient vectors **c, l, u** of the augmented form are then defined by:
$$\begin{align}
    \mathbf{c} = \begin{pmatrix} \text{mc}_{Conv} \\ \text{mc}_{Wind} \\ \text{mc}_{Conv} \\ \text{mc}_{Wind} \\ - \text{MWP}_{C1} \\ - \text{MWP}_{C2} \\ - \text{MWP}_{C1} \\ - \text{MWP}_{C2} \end{pmatrix}, \qquad \mathbf{l} = \begin{pmatrix} 0\\ 0\\0\\0\\0\\0\\0\\0\end{pmatrix}, \qquad \mathbf{u} = \begin{pmatrix} q_{Conv, 1} \\ q_{Wind, 1} \\ q_{Conv, 2} \\ q_{Wind, 2} \\ \overline{D}_{C1, 1} \\ \overline{D}_{C2, 1} \\ \overline{D}_{C1, 2} \\ \overline{D}_{C2, 2} \end{pmatrix} \tag{3B}
\end{align}$$

We can define these vectors "manually" e.g. using numpy's ```hstack``` method:

In [ ]:
c = np.hstack([mc, mc, -MWP, -MWP])
l = np.zeros(8)
u = np.hstack([q, D̄, D̄])
pd.DataFrame({'c':c, 'l':l, 'u':u}) # show the vectors here

Finally, we need to deal with the variational constraints. In our case, we have one equilibrium constraint (2B) for each hour ($h$). This translates to the coefficient vector and matrix:

$$\begin{align}
    \mathbf{b}_{eq} = \begin{pmatrix} 0 \\ 0 \end{pmatrix}, \qquad \mathbf{A}_{eq} = \begin{pmatrix} 1 & 1 & 0 & 0 & -1 & -1 & 0 & 0 \\ 0 & 0 & 1 & 1 & 0 & 0 & -1 & -1 \end{pmatrix}. \tag{3C}
\end{align}$$

Note that the first row in $\mathbf{A}_{eq}$ picks out generation and demand components for hour $h=1$ and the second row for $h=2$ (when multiplied by $\mathbf{x}$). We can define these objects "manually" as well:

In [ ]:
b_eq = np.zeros(2)
b_eq 

In [ ]:
A_eq = np.array([[1, 1, 0, 0, -1, -1, 0, 0], [0, 0, 1, 1, 0, 0, -1, -1]])
A_eq

Now, we can pass the elements to the solver and get the solution:

In [ ]:
sol = optimize.linprog(c, A_eq = A_eq, b_eq = b_eq, bounds = np.vstack([l, u]).T)
sol

## 3. Model specification with ```lpBlock``` <a id='ModelLP'> </a>

With a relatively small model, the manual approach outlined above is manageable: However, for a large number of variables and/or larger sets/indices, this becomes a very tedious exercise. In particular, it becomes a large task to keep track of the ordering of variables and constraints along the way. The ```lpBlock``` provides a way to specify the model in terms of variable names and sets (defined as pandas indices). Beyond this, the class also automatically keeps track of the internal ordering of the vectors, it automatically keeps coefficient matrices sparse (important for large models), and it returns solutions that are also split into variable names and sets (instead of one large, stacked numpy array).

### 3.1. Fundamentals <a id='ModelLP_Fundamentals'> </a>

For the model example above, we have the following fundamentals:
* *Sets:* The model is defined using three fundamental sets: Hours ($h$), consumers ($c$), and generators ($id$).
* *Variables:* The model solves for two endogenous variables: Generation ($E_{id,h}$) and hourly demand ($E_{c,h}$). 
* *Parameters/coefficients:* Exogenous to the model, we include parameters: 
    1. Marginal costs $(\text{mc}_{id})$, 
    2. marginal willingness to pay $(\text{MWP}_c)$,
    3. generation capacities $(q_{id,h})$, and 
    4. maximum load ($\overline{D}_c$).
* *Variational constraints:* We have a single contraint, 'equilibrium', defined over hours ($\text{equilibrium}_h$).

Often, we will also work with a database that collects all of our data. We use a small home-made database package ```pyDbs``` for this purpose, as it has a few build-in features that are useful here.

#### Create database with fundamentals

In [ ]:
db = pyDbs.SimpleDB() # initialize empty database

*Add parameters that we defined in the manual section:*

In [ ]:
db['mc'] = mc
db['D̄']  = D̄
db['MWP']= MWP
db['q']  = q

*We can automatically define the sets that are used in the parameters by calling the auxiliary function ```pyDbs.readSets(db)```:*

In [ ]:
print('Symbols registered in the database: ', db.symbols.keys()) # The symbols in the database are stored as keys
                                                                 # in the central dictionary, accessed by db.symbols
print('Example symbol "q": \n\n', db['q'], '\n')  # The values associated with a symbol can be viewed using a dictionary-style syntax
pyDbs.readSets(db)  # This function adds the unique indices in each of the symbol series as separate symbols to the db
print('Symbols including the new sets: ', db.symbols.keys())
print('Values in the "id" set: ', db['id']) # Now we can access the index values using the same syntax

Before we proceed, we define a new set called an ```alias``` (borrowed from GAMS syntax). The idea behind an alias is that it provides a new name for the same index - essentially providing a copy of the set. In our case, we add an alias for the hourly set (as we will need it later):

In [ ]:
db.updateAlias(alias = [('h','h_constr')]) # define h_constr as a copy of the index 'h'.

#### Define domains for variables and constraints 

Before specifying the model, it is often useful to start by explicitly defining the domains for our variables and constraints (what sets are they defined over). We do not need this step, but it helps clarify things: 

In [ ]:
globalDomains = {'E':   pd.MultiIndex.from_product([db['id'], db['h']]),
                 'D': pd.MultiIndex.from_product([db['c'], db['h']]),
                 'equilibrium':  db['h_constr']}
globalDomains

Note that we state that the constraint 'equilibrium' is defined over the aliased set ```h_constr```. It is, generally, a good convention to always define constraints over aliased sets, as we will often need to distinguish between e.g. what hours refer to the variable in a constraint ($h$) and what refers to the hours in the constraint (```h_constr```). In the context of the present model, the equilibrium constraint is only affected by generation and demand occuring within the same hour, so we can use ```h_constr``` to index the rows of the $A_{eq}$ matrix and let $h$ be part of the index for the columns in the matrix. If the row index doesn't match the column hour index, we know that element in the matrix should be zero.

### 3.2. Adding components to the ```c``` vector <a id='ModelLP_componentsStart'> </a>

Now that we have defined the fundamentals used in the model (sets, parameters, and domains for variables and constraints), we are ready to specify the model structure. We start by initializing an empty ```lpBlock``` instance with the structure of our 'globalDomains' from above:

In [ ]:
block = lpCompiler.lpBlock(globalDomains = globalDomains) # initialize empty instance
print(block.parameters)    # view the structure of the block class. As of now, all vectors and matrices are undefined

Next, we add components to the ```c``` vector using the ```add_c``` method. In our case, we want to add the parameter $\text{mc}_{id}$, but we want to repeat the marginal costs for all hours e.g. to match the domains in $E_{id,h}$. Expanding one symbol to cover a larger domain is called *broadcasting*, and we can do this calling a simple homemade function ```pyDbs.broadcast```:

In [ ]:
print('Original "mc" structure: \n\n', db['mc'], '\n') # this is our original mc data
# broadcast mc to match the larger index from Generation
print('Broadcasted structure: \n\n', pyDbs.broadcast(db['mc'], block.globalDomains['E'])) 

Similarly, we need a broadcasted version of the marginal willingness to pay for the $D$ variable:

In [ ]:
print('Original "MWP" structure: \n\n', db['MWP'], '\n') # this is our original MWP data
# broadcast MWP to match the larger index from D
print('Broadcasted structure: \n\n', -pyDbs.broadcast(db['MWP'], block.globalDomains['D']) )

We can now specify the ```c``` block as follows:

In [ ]:
block.add_c(varName = 'E', value = pyDbs.broadcast(db['mc'], block.globalDomains['E']), component = None, conditions = None)
block.add_c(varName = 'D', value = -pyDbs.broadcast(db['MWP'], block.globalDomains['D']), component = None, conditions = None)

> *Remark 1: The ```component = None, conditions = None``` are default options that can be left out here. The ```component``` option is used if we want to add multiple arguments to the 'c'-vector for the specific variable. The ```conditions``` option is used if we want to perform adjustments to the parameter added as ```value```. For instance, maybe this argument only holds for a subset of generators. You can see an example of this in the model class ```mBasicPH```.*

> *Remark 2: The method ```add_c``` processes the input ```value``` differently depending on the type of input. If the input is a scalar, it automatically tries to broadcast this number to the domains in ```globalDomains[varName]```. If the input is iterable (e.g. a list or a tuple), it takes the sum over all of the inputs.*

> *Remark 3: If the model specification does not explicitly provide a value for $\mathbf{c}$ for a specific variable (or even just a subset of the domains for a variable), the ```lpBlock``` automatically fills in zeros before solving the model.*  

In [ ]:
# unpack and view the updated 'c' vector
print('The updated "c" vector in our block: \nmc: \n', block.parameters['c'][('E', None)])
print('MWP: \n', block.parameters['c'][('D', None)])

### 3.3. Adding components to the ```l, u``` vectors

The methods for adding lower and upper bounds to the model are basically the same (but named ```add_l, add_u``` instead). If we do not specify a lower bound for a variable (or even a subset of the domains for a variable), the ```lpBlock``` compiler defaults to using zero in this case. For the upper bound, the default is to use ```np.inf``` (unbounded). For our model, where the lower bound is zero everywhere, this means that we can entirely skip specifying any ```l``` components.

In [ ]:
# block.add_l(varName = 'E', value = 0) # we do not need to specify these lower bounds
# block.add_l(varName = 'D', value = 0) 

For the upper bounds, we add:

In [ ]:
block.add_u(varName = 'E', value = db['q'])
block.add_u(varName = 'D', value = pyDbs.broadcast(db['D̄'], block.globalDomains['D']))

In [ ]:
# unpack and view the updated 'u' vector
print('The updated "u" vector in our block: \nq: \n', block.parameters['u'][('E', None)])
print('D̄: \n', block.parameters['u'][('D', None)])

### 3.4. Adding components to the ```b_eq, b_ub``` vector

The methods for adding parameters to the ```b_eq, b_ub``` vectors are essentially similar to the other vector-methods described above. The only difference is essentially that we use the keyword argument ```constrName``` instead of ```varName``` when we add something. Furthermore, as was the case when we specified the lower bound, the ```lpBlock``` automatically fills in some default values. As our model only includes zeros in ```b_eq``` (the default value), we simply add:

In [ ]:
block.add_b_eq(constrName = 'equilibrium') 

In [ ]:
# unpack and view the updated 'b_eq' vector
print('The updated "b_eq" vector in our block: \nb_eq: \n', block.parameters['b_eq']['equilibrium'])

### 3.5. Adding components to the ```A_eq, A_ub``` matrices <a id='ModelLP_componentsEnd'> </a>

Specifying coefficient matrices ```A_eq, A_ub``` is a bit more complicated, the reason simply being that we need to keep track of both variable and constraint indices. 

Recall from equation (3C) (repeated here for convenience) that the matrix consists of ones for each $E_{id,h}$ for each row $\text{equilibrium}_{h\_constr}$ where $h=h\_constr$

$$\begin{align}
    \mathbf{A}_{eq} = \begin{pmatrix} 1 & 1 & 0 & 0 & -1 & -1 & 0 & 0 \\ 0 & 0 & 1 & 1 & 0 & 0 & -1 & -1 \end{pmatrix}. 
\end{align}$$


We specify this by first creating a suitable pandas series:

In [ ]:
symbol = pd.Series(1, index = block.globalDomains['E'])
symbol # this includes all the ones required in the A_eq matrix

The small homemade function ```base.appIndexWithCopySeries(symbol, copyLevel, newLevel)``` creates a version of ```symbol``` that has a copy of the index level ```copyLevel``` called ```newLevel```. In this case, we specify that the ones in the symbol has to be for all cases where $h = h\_constr$:

In [ ]:
equiGen_Series = base.appIndexWithCopySeries(symbol, 'h','h_constr')
equiGen_Series

The important part to notice here is that if we "unstack" this to a dataframe with columns defined as the constraint index, we arrive at the relevant ```A_eq``` matrix (this is essentially what the compiler does under the hood):

In [ ]:
equiGen_Series.unstack(block.globalDomains['equilibrium'].names).fillna(0)

Similarly, for the demand part, we add:

In [ ]:
equiDem_Series = base.appIndexWithCopySeries(pd.Series(-1, index = block.globalDomains['D']), 'h','h_constr')
equiDem_Series

We add these to the components in ```A_eq``` using the syntax:

In [ ]:
block.add_A_eq(constrName = 'equilibrium', varName = 'E', value = equiGen_Series)
block.add_A_eq(constrName = 'equilibrium', varName = 'D', value = equiDem_Series)

In [ ]:
# unpack and view the updated 'A_eq' vector
print('The updated "A_eq" vector in our block: \nA_eq - E: \n', block.parameters['A_eq'][('equilibrium', 'E', None)])
print('A_eq - D : \n', block.parameters['A_eq'][('equilibrium', 'D', None)])

### 3.6. High level summary of code <a id='ModelLP_summary'> </a>

As this is meant as a documentation of how to use the code, the previous subsections may make it look incredibly difficult to apply this framework. So, to mitigate this, we end with a summary of what we need to specify this model from scratch:

*Create database (not necessary, but nice to have all data in one place):*

In [ ]:
db = pyDbs.SimpleDB(alias = [('h','h_constr')]) # initialize db with 'h_constr' as an alias of 'h'
db['mc']  = mc
db['D̄']   = D̄
db['MWP'] = MWP
db['q']   = q
pyDbs.readSets(db)

*Specify domains for variables and constraints:*

In [ ]:
globalDomains = {'E':   pd.MultiIndex.from_product([db['id'], db['h']]),
                 'D': pd.MultiIndex.from_product([db['c'], db['h']]),
                 'equilibrium':  db['h_constr']}

*Create lpBlock instance with model structure*

In [ ]:
block = lpCompiler.lpBlock(globalDomains = globalDomains) # initialize empty instance
# Add c-vector:
block.add_c(varName = 'E', value = pyDbs.broadcast(db['mc'], block.globalDomains['E']))
block.add_c(varName = 'D', value = -pyDbs.broadcast(db['MWP'], block.globalDomains['D']))
# add bounds:
block.add_u(varName = 'E', value = db['q'])
block.add_u(varName = 'D', value = pyDbs.broadcast(db['D̄'], block.globalDomains['D']))
# add constraints:
block.add_b_eq(constrName = 'equilibrium') 
block.add_A_eq(constrName = 'equilibrium', varName = 'E', value = base.appIndexWithCopySeries(pd.Series(1, index = block.globalDomains['E']), 'h','h_constr'))
block.add_A_eq(constrName = 'equilibrium', varName = 'D', value = base.appIndexWithCopySeries(pd.Series(-1, index = block.globalDomains['D']), 'h','h_constr'))

*Compile model and pass to solver by running:*

In [ ]:
# sol = optimize.linprog(**block())
# sol

*Export the model and database to use in the next part:*

In [ ]:
with open(os.path.join(d['data'], 'blockPartI'), "wb") as file:
    pickle.dump(block, file)
with open(os.path.join(d['data'], 'blockPartI_db'), "wb") as file:
    pickle.dump(db, file)

*Proceed to [**Part II**](./lpCompiler_PartII.ipynb) to check out the compilation phase.*